## Project Scenario
----


Compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, you need to transform the data and store it in USD, GBP, EUR, and INR per the exchange rate information made available to you as a CSV file. You should save the processed information table locally in a CSV format and as a database table. Managers from different countries will query the database table to extract the list and note the market capitalization value in their own currency.


In [1]:
import pandas as pd 
import numpy as np 
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import sqlite3

url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = ['Name','MC_USD_Billion']
csv_path = './Largest_banks_data.csv'
table_name = 'Largest_banks'
log_file = './code_log.txt'

In [2]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the code execution to a log file.'''
    now = datetime.now()
    timestamp = now.strftime('%Y-%h-%d-%H:%M:%S')
    with open(log_file,'a') as f:
        f.write(timestamp+' : '+message+'\n')


log_progress('Preliminaries complete. Initiating ETL process')

In [3]:
def extract(url, table_attribs):
    ''' This function aims to extract the required information from the website and save it to a data frame.'''
    df = pd.DataFrame(columns=table_attribs)

    data = requests.get(url)
    soup = BeautifulSoup(data.content,'html.parser')
    pages = soup.find('tbody')
    rows = pages.find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col) != 0:
            dict_ = {
                'Name': col[1].text[:-1],
                'MC_USD_Billion': float(col[2].text[:-1])
            }
            df_ = pd.DataFrame(dict_,index=[0])
            df = pd.concat([df,df_],ignore_index=True)

    return df


log_progress('Data extraction complete. Initiating Transformation process')

In [4]:
def transform(df):
    ''' This function accesses the CSV file for exchange rate information, and adds three columns to the data frame, 
    each containing the transformed version of Market Cap column to respective currencies'''
    
    exchange_rate = {'EUR':0.93,'GBP':0.8,'INR':82.95}
    
    df['MC_EUR_Billion'] = [np.round(x*exchange_rate['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_GBP_Billion'] = [np.round(x*exchange_rate['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*exchange_rate['INR'],2) for x in df['MC_USD_Billion']]

    return df


log_progress('Data transformation complete. Initiating Loading process')

In [5]:
def load_to_csv(df, csv_path):
    ''' This function saves the final data frame as a CSV file in the provided path.'''
    df.to_csv(csv_path,index=True)


log_progress('Data saved to CSV file')

In [6]:
connection = sqlite3.connect('Banks.db')
log_progress('SQL Connection initiated')

def load_to_db(df, connection, table_name):
    ''' This function saves the final data frame to a database table with the provided name.'''
    df.to_sql(table_name,connection,if_exists='replace',index=False)


log_progress('Data loaded to Database as a table, Executing queries')

In [7]:
def run_query(sql_statement, connection):
    ''' This function runs the query on the database table and prints the output on the terminal. '''
    query = pd.read_sql(sql_statement,connection)
    print(query)


log_progress('Process Complete')

In [8]:
extracted_data = extract(url,table_attribs)
transformed_data = transform(extracted_data)
load_data_csv = load_to_csv(transformed_data,csv_path)
load_data_db = load_to_db(transformed_data,connection,table_name)

In [9]:
output1 = run_query('Select * from Largest_banks',connection)
output1 = run_query('Select avg(MC_GBP_Billion) from Largest_banks',connection)
output1 = run_query('Select Name from Largest_banks order by MC_USD_Billion desc limit 5',connection)

                                       Name  MC_USD_Billion  MC_EUR_Billion  \
0                            JPMorgan Chase          432.92          402.62   
1                           Bank of America          231.52          215.31   
2   Industrial and Commercial Bank of China          194.56          180.94   
3                Agricultural Bank of China          160.68          149.43   
4                                 HDFC Bank          157.91          146.86   
5                               Wells Fargo          155.87          144.96   
6                         HSBC Holdings PLC          148.90          138.48   
7                            Morgan Stanley          140.83          130.97   
8                   China Construction Bank          139.82          130.03   
9                             Bank of China          136.81          127.23   

   MC_GBP_Billion  MC_INR_Billion  
0          346.34        35910.71  
1          185.22        19204.58  
2          155.65     

In [10]:
connection.close()
log_progress('Server Connection closed')